In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("ProyectoSparkie-PalabrasDescriptivas") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/12/07 23:39:05 WARN Utils: Your hostname, DESKTOP-6OAF9F9, resolves to a loopback address: 127.0.1.1; using 172.28.59.214 instead (on interface eth0)
25/12/07 23:39:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/07 23:39:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Aquí vamos a cargar un RDD que ya hicimos con las frecuencias de las palabras en un documento

import ast

ruta_freq = "../data/processed/frecuencias_rdd"

rdd_raw = sc.textFile(ruta_freq)

def parse_f(line):
    """Convierte el string guardado en tupla Python"""
    return ast.literal_eval(line)

rdd_freq = rdd_raw.map(parse_f)

#Comprobar que todo esté en orden:
print("- rdd_freq cargado correctamente")
print("Ejemplo:", rdd_freq.take(5))


- rdd_freq cargado correctamente


[Stage 0:>                                                          (0 + 1) / 1]

Ejemplo: [(('Romeo_and_Juliet_by_William_Shakespeare.txt', 'tragedy'), 1), (('Romeo_and_Juliet_by_William_Shakespeare.txt', 'romeo'), 316), (('Romeo_and_Juliet_by_William_Shakespeare.txt', 'william'), 1), (('Romeo_and_Juliet_by_William_Shakespeare.txt', 'shakespeare'), 1), (('Romeo_and_Juliet_by_William_Shakespeare.txt', 'contents'), 1)]


In [3]:
#Aquí extraemos los libros de la lista disponible que ya tenemos
# Extraer solo los nombres de libro
libros_disponibles = rdd_freq.map(lambda x: x[0][0]).distinct().collect()

print(f"- Libros detectados: {len(libros_disponibles)}")
print(libros_disponibles[:10])



[Stage 2:=============================>                             (1 + 1) / 2]

- Libros detectados: 99
['Moby_Dick;_Or,_The_Whale_by_Herman_Melville.txt', 'Peter_Pan___by_J._M._Barrie.txt', 'Thus_Spake_Zarathustra__A_Book_for_All_and_None_by_Friedrich_Wilhelm_Nietzsche.txt', 'A_Christmas_Carol_by_Charles_Dickens.txt', 'The_Tragical_History_of_Doctor_Faustus_by_Christopher_Marlowe.txt', "Grimms'_Fairy_Tales_by_Jacob_Grimm_and_Wilhelm_Grimm.txt", 'My_Life_—_Volume_1_by_Richard_Wagner.txt', 'The_Count_of_Monte_Cristo_by_Alexandre_Dumas_and_Auguste_Maquet.txt', 'The_divine_comedy_by_Dante_Alighieri.txt', 'Middlemarch_by_George_Eliot.txt']


In [4]:
# Buscamos y validamos que el libro exista para que regrese error si no existe.

def buscar_libro(nombre_libro, libros_disponibles):
    """
    Busca un libro en la lista.
    
    Parámetros:
    - nombre_libro: nombre del libro
    - libros_disponibles: lista de todos los libros
    
    Retorna:
    - nombre_completo del libro si existe
    - None si no existe
    """
    # Búsqueda exacta
    nombre_lower = nombre_libro.lower()
    
    for libro in libros_disponibles:
        if libro.lower() == nombre_lower:
            return libro
    
    # Búsqueda parcial
    coincidencias = [libro for libro in libros_disponibles 
                     if nombre_lower in libro.lower()]
    
    if len(coincidencias) == 1:
        print(f"Libro encontrado: {coincidencias[0]}")
        return coincidencias[0]
    elif len(coincidencias) > 1:
        print(f"Se encontraron {len(coincidencias)} libros que coinciden:")
        for i, libro in enumerate(coincidencias[:5], 1):
            print(f"   {i}. {libro}")
        if len(coincidencias) > 5:
            print(f"   ... y {len(coincidencias) - 5} más")
        print("\n Por favor, sea más específico con el nombre.")
        return None
    else:
        print(f"Error: El libro '{nombre_libro}' no existe en la colección.")
        print("\n Sugerencias (libros similares):")
        # Buscar libros con palabras similares
        palabras = nombre_lower.split()
        sugerencias = []
        for libro in libros_disponibles:
            libro_lower = libro.lower()
            if any(palabra in libro_lower for palabra in palabras):
                sugerencias.append(libro)
        
        for i, libro in enumerate(sugerencias[:5], 1):
            print(f"   {i}. {libro}")
        
        return None

# Prueba de la función
print("Prueba de búsqueda:")
prueba = buscar_libro("odyssey", libros_disponibles)

Prueba de búsqueda:
Libro encontrado: The_Odyssey_by_Homer.txt


In [8]:
#Aquí tenemos la función principal, la que se encarga de retornar qué palabras se repiten más  

def palabras_descriptivas(libro, M=10):
    """
    Retorna las M palabras más representativas del libro.
    Basado en frecuencia.
    Si el libro no existe → error.
    """

    libro_ok = buscar_libro(libro, libros_disponibles)
    if libro_ok is None:
        return None

    print(f"\nPalabras clave de: {libro_ok}\n")

    # Filtrar solo ese documento
    palabras = (rdd_freq
        .filter(lambda x: x[0][0] == libro_ok)   # x[0] = (doc,palabra)
        .map(lambda x: (x[0][1], x[1]))          # quedarnos con (palabra , freq)
        .sortBy(lambda x: x[1], ascending=False)
        .take(M)
    )

    return palabras


In [11]:
#Ejemplo 1 con libro que sabemos que existe

ejemplo = palabras_descriptivas("Odyssey", M=15)

if ejemplo:
    print("Palabras que lo describen:\n")
    for i, (palabra, freq) in enumerate(ejemplo,1):
        print(f"{i}. {palabra}  -> {freq} apariciones")


Libro encontrado: The_Odyssey_by_Homer.txt

Palabras clave de: The_Odyssey_by_Homer.txt



[Stage 13:>                                                         (0 + 1) / 1]

Palabras que lo describen:

1. ulysses  -> 655 apariciones
2. one  -> 571 apariciones
3. said  -> 483 apariciones
4. house  -> 413 apariciones
5. would  -> 385 apariciones
6. man  -> 347 apariciones
7. men  -> 335 apariciones
8. went  -> 301 apariciones
9. come  -> 290 apariciones
10. telemachus  -> 282 apariciones
11. son  -> 280 apariciones
12. upon  -> 269 apariciones
13. tell  -> 263 apariciones
14. see  -> 261 apariciones
15. may  -> 261 apariciones


In [10]:
print("\nPrueba con libro inexistente:")
palabras_descriptivas("Harry Potter", M=10)



Prueba con libro inexistente:
Error: El libro 'Harry Potter' no existe en la colección.

 Sugerencias (libros similares):
